# Deep Learning Lab 2

## Imports

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

!cp /content/drive/MyDrive/DL/L2/MAMe_data_256.zip .
!cp /content/drive/MyDrive/DL/L2/MAMe_metadata.zip .


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!unzip -qq MAMe_data_256.zip
!unzip -qq MAMe_metadata.zip
!rm MAMe_data_256.zip
!rm MAMe_metadata.zip
!rm MAMe_dataset.csv
!cp /content/drive/MyDrive/DL/L2/MAMe_dataset.csv .


replace data_256/100019.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
replace data_256/100033.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace MAMe_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import random
import cv2
from PIL import Image
from skimage import io
from tensorflow.keras import utils
from tensorflow.keras.utils import Sequence
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from skimage.transform import rotate, rescale
from keras.models import Sequential, Model 
from sklearn.preprocessing import LabelEncoder


## Constants

In [ ]:
FILE_TEST = "MAMe_toy_dataset.csv"
FILE_TRAIN = "MAMe_dataset.csv"
NUM_CLASSES = 29


## Pre process

In [ ]:
def split_data(train, test, validation ):
    names_images_train = train[:,0]
    names_images_test = test[:,0]
    names_images_val = validation[:,0]
    y_train = train[:,1]
    y_test = test[:,1]
    y_val = validation[:,1]
    return (names_images_train, names_images_test, names_images_val, y_train, y_test, y_val)

def transform_labels(labels_dict, data):
    """
    Transform data strings to numerical labels
    @param labels_dict: dictionary with the labels
    @param data: the data to transform
    @return: the transformed data
    """
    y_train, y_test, y_val = [], [], []
    for x in data[3]:
        y = np.zeros(NUM_CLASSES)
        y[labels_dict[x]] = 1
        y_train.append(y)
    for x in data[4]:
        y = np.zeros(NUM_CLASSES)
        y[labels_dict[x]] = 1
        y_test.append(y)
    for x in data[5]:
        y = np.zeros(NUM_CLASSES)
        y[labels_dict[x]] = 1
        y_val.append(y)
    return (np.array(y_train), np.array(y_test), np.array(y_val))

def load_data(file):
    """
    Split data from csv file in three parts: train, test and validation
    """
    file_labels = pd.read_csv(file).to_numpy()
    train = file_labels[file_labels[:, 4] == "train"]
    test = file_labels[file_labels[:,4] == "test"]
    validation = file_labels[file_labels[:,4] == "val"]
    return split_data(train, test, validation)
    #print("Train size: ", len(train), "\nTest size: ", len(test), "\nValidation size: ", len(validation))

def labels_to_dict(labels):
    my_labels = {}
    for i in range(len(labels)):
        my_labels[labels[i,1]] = labels[i,0]
    return my_labels


def load_labels():
    """
    Load labels from csv file
    @return numpy array of labels
    """
    labels = pd.read_csv("MAMe_labels.csv", header=None).to_numpy()
    return labels_to_dict(labels)
    
def load_images(data):
    """
    Load images from name of given in the data
    @param data: numpy array with names of images
    @return numpy array with images
    """
    data_x = []
    for name in data:
        img = io.imread("data_256/" + name)
        data_x.append(img)
 
    return np.array(data_x)



In [ ]:
def get_valSet(x_test, y_test, rotation=0, horizontal_flip=False):
    """
    Gets the validation set.
    :param x_test: test set
    :param y_test: test labels
    :return: validation set
    """
    val_data_gen_args = dict(rescale = None,
                      samplewise_center=True,
                      samplewise_std_normalization=True,
                      rotation_range=rotation,
                      horizontal_flip=horizontal_flip)
   
    val_datagen = ImageDataGenerator(val_data_gen_args)
    val_set = val_datagen.flow(x_test, y_test, batch_size=BATCH_SIZE)
    return val_set


## Data Augmentation

In [ ]:
def data_augmentation(x_names, y_names, val):
  x = []
  y = []
  print(x_names.shape, y_names.shape)
  #seed = random.randint(0,50)
  #random.Random(seed).shuffle(x_names)
  #random.Random(seed).shuffle(y_names)
  for i in range(int(len(x_names)*val)):
    y.append(y_names[i])
    image = Image.open('/content/data_256/'+ x_names[i])
    image = image.rotate(random.randint(0, 359))
    name_dest = x_names[i].split('.')[0] + "_rotated.jpg"
    image.save('/content/data_256/' + name_dest)
    x.append(name_dest)
  x_train = np.concatenate((x_names, x))
  y_train = np.concatenate((y_names, y))
  return x_train, y_train

## Data Generator

In [ ]:
class DataGenerator(Sequence):

  def __init__(self, x_names, y_names, batch_size):
    
    self.x_values = x_names.copy()
    self.y_values =   y_names.copy()
    self.num_imgs = len(x_names)
    self.batch_size = batch_size

  def __getitem__(self, index):

    # position of the batch in the sequence

    aux_index = index * self.batch_size

    x = [] # numpy array with shape (batch_size, input_height, input_width, input_channel)
    y = [] # numpy array with shape (batch_size, num_classes)
    

    for _ in range(self.batch_size):

      img_name = self.x_values[aux_index]
      img = cv2.imread('/content/data_256/' + img_name)
      x.append(img)
      y.append(self.y_values[aux_index])
      aux_index += 1

    x = np.array(x)
    y = np.array(y)
    return x, y

  def on_epoch_end(self):
    num = random.randint(0,1000)
    random.Random(num).shuffle(self.x_values)
    random.Random(num).shuffle(self.y_values)

  def __len__ (self):
    # return the number of batches the generator can produce
    return (self.num_imgs) // self.batch_size

In [ ]:
def get_valSet(x_test, y_test, rotation=False):
    """
    Gets the validation set.
    :param x_test: test set
    :param y_test: test labels
    :return: validation set
    """
    val_data_gen_args = dict(rescale = None,
                     samplewise_center=True,
                     samplewise_std_normalization=True)
   
    val_datagen = ImageDataGenerator(val_data_gen_args)
    val_set = val_datagen.flow(x_test, y_test, batch_size=BATCH_SIZE)
    return val_set

## Feature Extraction

## Fine Tunning

In [ ]:
def train_model_fine(data_x, data_val, data_y ,lr= 0.001):
  img_width, img_height = 256, 256
  model = applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
  #model.trainable = True
  for layer in model.layers[:10]:
    layer.trainable = False

  #Adding custom Layers 
  x = model.output
  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(4096, activation="relu")(x)
  x = keras.layers.Dropout(0.2)(x)
  x = keras.layers.Dense(1024, activation="relu")(x)
  predictions = keras.layers.Dense(29, activation="softmax")(x)

  # creating the final model 
  model_final = Model(model.input, predictions)
  early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto')
  model_final.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
  history = model_final.fit(
        data_x, data_y,
        epochs = 64,
        validation_data = data_val,
        batch_size=BATCH_SIZE,
        shuffle=True,
        callbacks = [early])#,checkpoint])
  return history, model_final

## Main

In [ ]:
DATA_AUGMENTATION = False
BATCH_SIZE = 128

In [ ]:
data = load_data(FILE_TRAIN) # Data is splitted
x_train = data[0]
labels_dict = load_labels()
y_vals = transform_labels(labels_dict, data)
y_train = y_vals[0]
if DATA_AUGMENTATION:
  x_train, y_train = data_augmentation(x_train, y_train, 0.2)
#data_train = DataGenerator(x_train, y_train, BATCH_SIZE)
#data_val = DataGenerator(data[2], y_vals[2], BATCH_SIZE)
print(y_train.shape, y_vals[2].shape)


(20300, 29) (1450, 29)


### Fine Tunning

In [ ]:
#print(x_train[-10])
data_train = load_images(data[0])

data_val = get_valSet(load_images(data[2]), y_vals[2])


In [ ]:
model_hist, model = train_model_fine(data_train, data_val, y_vals[0])
#model_hist, model = train_model_fine(data_train, data_val, y_train)


Epoch 1/64
159/159 [==============================] - ETA: 0s - loss: 4.1365 - accuracy: 0.0440

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


159/159 [==============================] - 209s 1s/step - loss: 4.1365 - accuracy: 0.0440 - val_loss: 3.1228 - val_accuracy: 0.0800
Epoch 2/64
159/159 [==============================] - 191s 1s/step - loss: 2.3674 - accuracy: 0.2673 - val_loss: 1.9043 - val_accuracy: 0.4034
Epoch 3/64
159/159 [==============================] - 193s 1s/step - loss: 1.7163 - accuracy: 0.4505 - val_loss: 1.6469 - val_accuracy: 0.4869
Epoch 4/64
159/159 [==============================] - 193s 1s/step - loss: 1.4706 - accuracy: 0.5283 - val_loss: 1.5359 - val_accuracy: 0.5028
Epoch 5/64
159/159 [==============================] - 193s 1s/step - loss: 1.3198 - accuracy: 0.5691 - val_loss: 1.3365 - val_accuracy: 0.5690
Epoch 6/64
159/159 [==============================] - 193s 1s/step - loss: 1.2202 - accuracy: 0.5993 - val_loss: 1.4233 - val_accuracy: 0.5462
Epoch 7/64
159/159 [==============================] - 193s 1s/step - loss: 1.1522 - accuracy: 0.6263 - val_loss: 1.3548 - val_accuracy: 0.5800
Epoch 8/64

## RESULTS

In [ ]:
def plot_training(history):
    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt

    #Accuracy plot
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','val'], loc='upper left')
    plt.title('Training and validation accuracy')
    plt.savefig('fine_tuning_accuracy.pdf')
    plt.close()
    #Loss plot
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','val'], loc='upper left')
    plt.title('Training and validation loss')
    plt.savefig('fine_tuning_loss.pdf')
    plt.close()
plot_training(model_hist)

## Predicts

In [ ]:
x_test, y_test = data[1], y_vals[1]
correct = 0
for name_img, result in zip(x_test, y_test):
    img = cv2.imread('/content/data_256/' + name_img)
    img = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
    img = img[np.newaxis, :, :,:]
    prediction = model.predict(img)
    if np.argmax(prediction) == np.argmax(result):
      correct += 1
print(correct/len(x_test))


0.4387175065465926


## Cells for reset folders

This cells are for reset folders and clear the images added using data Augmentation

In [ ]:
!rm -r data_256/*.jpg
!cp /content/drive/MyDrive/DL/L2/MAMe_data_256.zip .
!unzip MAMe_data_256.zip